<a href="https://colab.research.google.com/github/Nisarg-1406/Python-Libraries-Concept/blob/main/sparklearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Executed as sparklearning file in Databricks community edition notebook 
# Course - https://www.udemy.com/course/spark-and-python-for-big-data-with-pyspark/

## TOPICS
# SECTION 8 - SPARK DATAFRAME BASICS
# SECTION 9 - WALMART DATA FOR SPARK DATAFRAME PROJECT 
# SECTION 10 - THEORY ABOUT MACHINE LEARNING WITH PYSPARK
# SECTION 11 - MACHINE LEARNING WITH PYSPARK (MLib)
# SECTION 12 - LOGISTIC REGRESSION - CLASSIFICATION
# SECTION 13 - TREE METHODS
# SECTION 14 - K MEANS AND CLUSTERING
# SECTION 15 - RECOMMENDER SYSTEM
# SECTION 16 - NATURAL LANGUAGE PROCESSING

## Could not do section 17 - spark streaming with python as it inlcudes the connection of ubuntu with virtual box in which my jupyter notebook is not supporting

In [ ]:
## SECTION 8 - SPARK DATAFRAME BASICS

# Start spark session 
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [ ]:
df1 = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/people.json")

In [ ]:
df1.show() #Empty values are replaced by null

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [ ]:
df1.printSchema() # As it is method, therefore to have the ()

root
-- age: long (nullable = true)
-- name: string (nullable = true)

In [ ]:
df1.columns

Out[9]: ['age', 'name']

In [ ]:
df1.describe() #Summary of Dataframe. So it calls a dataframe of column summary which has some string in it and column age which also has some string in it. 

Out[10]: DataFrame[summary: string, age: string, name: string]

In [ ]:
df1.describe().show() # To entire see the dataframe. 

+-------+------------------+-------+
summary| age| name|
+-------+------------------+-------+
 count| 2| 3|
 mean| 24.5| null|
 stddev|7.7781745930520225| null|
 min| 19| Andy|
 max| 30|Michael|
+-------+------------------+-------+

In [ ]:
# If the data is not very nice or not sure from where the data is being taken, we need to clarify the schema of the data
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
# We are creating the list in which we are taking StructField in which we are taking name as 'age' which has the instance of IntegerType OR age is of datatype integer (age was of long type in the start) and 'True' is passed beacuse if there is null value then if we do not pass true then it will give the error. Same with other StructField of name col
data_schema = [StructField('age', IntegerType(), True), StructField('name', StringType(), True)]

In [ ]:
# Putting the data schema into final struct
final_struct = StructType(fields=data_schema) 

In [ ]:
df1 = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/people.json",schema=final_struct)

In [ ]:
df1.printSchema()

root
-- age: integer (nullable = true)
-- name: string (nullable = true)

In [ ]:
# To get the column object
type(df1['age'])

Out[19]: pyspark.sql.column.Column

In [ ]:
# Want to get dataframe with single column - we use the select() method
df1.select("age").show()
type(df1.select("age")) # Dataframe

+----+
 age|
+----+
null|
 30|
 19|
+----+

Out[23]: pyspark.sql.dataframe.DataFrame

In [ ]:
# To get the list of row objects
df1.head(3)[1]

# There is row object and col object is the sparks ability to read from a distributed source and map that out to the distributed computing 

Out[26]: Row(age=30, name='Andy')

In [ ]:
# Select multiple columns - It returns the dataframes of 2 columns in this case
df1.select(['age','name']).show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [ ]:
# To create the new column - we use withColumn function where first parameter is the name of the column and second parameter is the existing col -> It is column.Column datatype. ALSO THIS CHANGES ARE NOT PERMENANT ON THE ORIGINAL DATAFRAME.  
df1.withColumn('newage', df1['age']).show()

df1.withColumn('double_age', df1['age']*2).show()

+----+-------+------+
 age| name|newage|
+----+-------+------+
null|Michael| null|
 30| Andy| 30|
 19| Justin| 19|
+----+-------+------+

+----+-------+----------+
 age| name|double_age|
+----+-------+----------+
null|Michael| null|
 30| Andy| 60|
 19| Justin| 38|
+----+-------+----------+

In [ ]:
# To change the name of the column 
df1.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
my_new_age| name|
+----------+-------+
 null|Michael|
 30| Andy|
 19| Justin|
+----------+-------+

In [ ]:
# sql temp view - createOrReplaceTempView() -> will create the temp view and if already temp view is created then it will replace the temp view 
df1.createOrReplaceTempView('people') # with this we have registered the sql temp view

In [ ]:
# As a string, passing the direct sql queries 
results = spark.sql("SELECT * FROM people")
results.show()

+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

In [ ]:
new_results = spark.sql("SELECT * FROM people WHERE age = 30")
new_results.show()

+---+----+
age|name|
+---+----+
 30|Andy|
+---+----+

In [ ]:
spark = SparkSession.builder.appName("ops").getOrCreate()

In [ ]:
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/appl_stock.csv", inferSchema = True, header = True) # header = True means very first row contains the column name 

In [ ]:
df1.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

In [ ]:
# To filter out the data - spark dataframes are built on top of the spark sql platforms 
# using sql
# df1.filter("Close < 500").show() # Shows all of the column
# df1.filter("Close < 500").select('open').show() # showing single column 
df1.filter("Close < 500").select(['open','close','low']).show() # To show multiple 

+------------------+------------------+------------------+
 open| close| low|
+------------------+------------------+------------------+
 213.429998| 214.009998|212.38000099999996|
 214.599998| 214.379993| 213.249994|
 214.379993| 210.969995| 210.750004|
 211.75| 210.58| 209.050005|
 210.299994|211.98000499999998|209.06000500000002|
212.79999700000002|210.11000299999998| 208.450005|
209.18999499999998| 207.720001| 206.419998|
 207.870005| 210.650002| 204.099998|
210.11000299999998| 209.43| 209.020004|
210.92999500000002| 205.93| 205.869999|
 208.330002| 215.039995| 207.240004|
 214.910006| 211.73| 209.500002|
 212.079994| 208.069996| 207.210003|
206.78000600000001| 197.75| 197.16|
202.51000200000001| 203.070002| 200.190002|
205.95000100000001| 205.940001| 202.580004|
 206.849995| 207.880005| 199.530001|
 204.930004| 199.289995| 198.699995|
 201.079996| 192.060003| 190.250002|
192.36999699999998| 194.729998|191.29999899999999|
+------------------+------------------+------------------+
only showing top 20 rows

In [ ]:
# This syntax is same as above in which we would be taking entire Close column and use python operator for comparision 
df1.filter(df1['close'] < 500).show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
2010-01-04| 213.429998| 214.499996|212.38000099999996| 214.009998|123432400| 27.727039|
2010-01-05| 214.599998| 215.589994| 213.249994| 214.379993|150476200|27.774976000000002|
2010-01-06| 214.379993| 215.23| 210.750004| 210.969995|138040000|27.333178000000004|
2010-01-07| 211.75| 212.000006| 209.050005| 210.58|119282800| 27.28265|
2010-01-08| 210.299994| 212.000006|209.06000500000002|211.98000499999998|111902700| 27.464034|
2010-01-11|212.79999700000002| 213.000002| 208.450005|210.11000299999998|115557400| 27.221758|
2010-01-12|209.18999499999998|209.76999500000002| 206.419998| 207.720001|148614900| 26.91211|
2010-01-13| 207.870005|210.92999500000002| 204.099998| 210.650002|151473000| 27.29172|
2010-01-14|210.11000299999998|210.45999700000002| 209.020004| 209.43|108223500| 27.133657|
2010-01-15|210.92999500000002|211.59999700000003| 205.869999| 205.93|148516900|26.680197999999997|
2010-01-19| 208.330002|215.18999900000003| 207.240004| 215.039995|182501900|27.860484999999997|
2010-01-20| 214.910006| 215.549994| 209.500002| 211.73|153038200| 27.431644|
2010-01-21| 212.079994|213.30999599999998| 207.210003| 208.069996|152038600| 26.957455|
2010-01-22|206.78000600000001| 207.499996| 197.16| 197.75|220441900| 25.620401|
2010-01-25|202.51000200000001| 204.699999| 200.190002| 203.070002|266424900|26.309658000000002|
2010-01-26|205.95000100000001| 213.710005| 202.580004| 205.940001|466777500| 26.681494|
2010-01-27| 206.849995| 210.58| 199.530001| 207.880005|430642100|26.932840000000002|
2010-01-28| 204.930004| 205.500004| 198.699995| 199.289995|293375600|25.819922000000002|
2010-01-29| 201.079996| 202.199995| 190.250002| 192.060003|311488100| 24.883208|
2010-02-01|192.36999699999998| 196.0|191.29999899999999| 194.729998|187469100| 25.229131|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
only showing top 20 rows

In [ ]:
# Suppose I wanted to grab all the rows having close price < 200$ and open price > 200$
# df1.filter(df1['Close'] < 200 and df1['Open'] > 200) # It will generate the value error - Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.
# df1.filter(df1['Close'] < 200 & df1['Open'] > 200) # This will also generate the error - python for java error

# To fix the above error we need to pass both of parameters sepeartly
df1.filter((df1['Close'] < 200) & (df1['Open'] > 200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
2010-01-22|206.78000600000001|207.499996| 197.16| 197.75|220441900| 25.620401|
2010-01-28| 204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
2010-01-29| 201.079996|202.199995|190.250002|192.060003|311488100| 24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+

In [ ]:
# df1.filter(df1['Low'] == 197.16).show()
result = df1.filter(df1['Low'] == 197.16).collect() #It shows list of row object
# result #It shows list of row object
row = result[0] # It provide the item

# Can be converted to the dictionary
row.asDict()

Out[55]: {'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [ ]:
## SECTION 8 -GROUPBY OPERATIONS

# Groupby allows to group out the rows based upon the column value. Aggregate combines the multiple rows of data as a single row o/p (Ex : to take the sum of bunch of inputs)
spark = SparkSession.builder.appName("aggrs").getOrCreate()

In [ ]:
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/sales_info.csv", inferSchema = True, header = True)

In [ ]:
df1.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

In [ ]:
df1.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [ ]:
df1.groupBy("Company") # Here we get the grouped data object at a location

Out[61]: <pyspark.sql.group.GroupedData at 0x7fd7daf56850>

In [ ]:
# df1.groupBy('Company').mean().show() # it gives the dataframe for mean(AVG) with Company col with string datatype and sales with double datatype
# df1.groupBy('Company').sum().show() # For sum 
df1.groupBy('Company').count().show() # For count

+-------+-----+
Company|count|
+-------+-----+
 APPL| 4|
 GOOG| 3|
 FB| 2|
 MSFT| 3|
+-------+-----+

In [ ]:
## SECTION 8 -AGGREGATE OPERATIONS

# We can call the generalise 'agg' method that would aggregate across all row in the dataframe. (.agg) method takes dictionary as the argument. Eg : It can be add all the sales without considering the company
df1.agg({'Sales' : 'sum'}).show() # For sum
df1.agg({'Sales' : 'max'}).show() # For max

+----------+
sum(Sales)|
+----------+
 4327.0|
+----------+

+----------+
max(Sales)|
+----------+
 870.0|
+----------+

In [ ]:
# We can add the combination of groupby and .agg
group_data = df1.groupBy("Company")
group_data.agg({'Sales' : 'max'}).show()

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

In [ ]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [ ]:
df1.select(countDistinct('sales')).show() #Count distinct number of sales
df1.select(avg('sales').alias('Average Sale')).show() # To give the name to the avg sale, we are using the function '.alias('Name to give')' 

+---------------------+
count(DISTINCT sales)|
+---------------------+
 11|
+---------------------+

+-----------------+
 Average Sale|
+-----------------+
360.5833333333333|
+-----------------+

In [ ]:
df1.select(stddev('sales')).show() # the name is stddev_samp(sales) which is not good looking and the number '250.08742410799007' is very unformated
# therefore will format the number 

from pyspark.sql.functions import format_number

sales_std = df1.select(stddev("Sales").alias('std'))
sales_std.select(format_number('std',2).alias('std')).show() # It will show upto 2 digits here in this case. 

+------------------+
stddev_samp(sales)|
+------------------+
250.08742410799007|
+------------------+

+------+
 std|
+------+
250.09|
+------+

In [ ]:
# if wanted to order by sales (Ascending order)
df1.orderBy('Sales').show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [ ]:
# To display in descending order
df1.orderBy(df1['sales'].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+

In [ ]:
## SECTION 8 -MISSING DATA 

spark = SparkSession.builder.appName("miss").getOrCreate()

In [ ]:
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/ContainsNull.csv", inferSchema = True, header = True)

In [ ]:
df1.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [ ]:
df1.na.drop().show() # to drop the row with any amount of missing data
df1.na.drop(thresh = 2).show() # This means the particular row must have atleast 2 not null values so that row is not dropped due to null in the col of that row
df1.na.drop(how = 'any').show() # to drop the row with any amount of missing data
df1.na.drop(how = 'all').show() # to drop the row with all amount of missing data i.e if all are null then to drop that row
df1.na.drop(subset = ['sales']).show() # it will drop the row if any value in sales is null, it will not matter that id and name at that row is null or not

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [ ]:
# To fill the value in the null place
df1.na.fill('FILL VALUE').show() # It is smart to fill the place where there is the string because in na.fill we passed a string and hence the name col where there was null value is filled with 'FILL VALUE'

df1.na.fill('NO NAME', subset = 'Name').show() # here we are passing the subset as parameter

df1.na.fill(0).show() # It is smart to fill the place where there is the number because in na.fill we passed a number and hence the sales col where there was null value is filled with '0'

+----+----------+-----+
 Id| Name|Sales|
+----+----------+-----+
emp1| John| null|
emp2|FILL VALUE| null|
emp3|FILL VALUE|345.0|
emp4| Cindy|456.0|
+----+----------+-----+

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| null|
emp2|NO NAME| null|
emp3|NO NAME|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| 0.0|
emp2| null| 0.0|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [ ]:
# To replace the null value with the mean value. 
from pyspark.sql.functions import mean

mean_value = df1.select(mean(df1['sales'])).collect()
mean_value # o/p - [Row(avg(sales)=400.5)]
mean_sales = mean_value[0][0] # 400.5
mean_sales

df1.na.fill('mean_sales', ['sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [ ]:
## SECTION 8 -DATE AND TIMESTAMPS

spark = SparkSession.builder.appName("dates").getOrCreate()

In [ ]:
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/appl_stock.csv", inferSchema = True, header = True) # header = True means very first row contains the column name 

df1.head(1)

Out[104]: [Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [ ]:
df1.select(['Date','open']).show()

+----------+------------------+
 Date| open|
+----------+------------------+
2010-01-04| 213.429998|
2010-01-05| 214.599998|
2010-01-06| 214.379993|
2010-01-07| 211.75|
2010-01-08| 210.299994|
2010-01-11|212.79999700000002|
2010-01-12|209.18999499999998|
2010-01-13| 207.870005|
2010-01-14|210.11000299999998|
2010-01-15|210.92999500000002|
2010-01-19| 208.330002|
2010-01-20| 214.910006|
2010-01-21| 212.079994|
2010-01-22|206.78000600000001|
2010-01-25|202.51000200000001|
2010-01-26|205.95000100000001|
2010-01-27| 206.849995|
2010-01-28| 204.930004|
2010-01-29| 201.079996|
2010-02-01|192.36999699999998|
+----------+------------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import dayofmonth, hour, dayofyear, month, year, weekofyear, format_number, date_format
df1.select(dayofmonth(df1['Date'])).show() # Due to mismatch of datatype, we need to pass df1['Date']
df1.select(hour(df1['Date'])).show() # hour of everyday starts at 0, rest of the hours are not shown - only first 20 rows is shown

+----------------+
dayofmonth(Date)|
+----------------+
 4|
 5|
 6|
 7|
 8|
 11|
 12|
 13|
 14|
 15|
 19|
 20|
 21|
 22|
 25|
 26|
 27|
 28|
 29|
 1|
+----------------+
only showing top 20 rows

In [ ]:
newdf = df1.withColumn("Year",year(df1['date']))

# if wanted to find the avg(close) for every year
result = newdf.groupBy("Year").mean().select(["Year","avg(Close)"])

In [ ]:
new_result = result.withColumnRenamed("avg(close)","Average Closing Price")
new_result.select(['year',format_number('Average Closing Price',2).alias("Avg Close")]).show()

+----+---------+
year|Avg Close|
+----+---------+
2015| 120.04|
2013| 472.63|
2014| 295.40|
2012| 576.05|
2016| 104.60|
2010| 259.84|
2011| 364.00|
+----+---------+

In [ ]:
## SECTION 9 - WALMART DATA FOR SPARK DATAFRAME PROJECT 

spark = SparkSession.builder.appName("dates").getOrCreate()

In [ ]:
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/walmart_stock.csv", inferSchema = True, header = True)

In [ ]:
for row in df1.head(5):
  print(row)

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)
Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)
Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)
Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)
Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)

In [ ]:
df1.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Date| Open| High| Low| Close| Volume| Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| null| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|2012-01-03|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max|2016-12-30| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [ ]:
# In this dataset there are many decimal points for mean and stddev. So we need to format the number to show upto 2 decimal places
from pyspark.sql.functions import format_number

In [ ]:
result = df1.describe()
result.select(result['summary'],format_number(result['open'].cast('float'),2).alias('Open'),format_number(result['close'].cast('float'),2).alias('close'),
                                format_number(result['High'].cast('float'),2).alias('High'),format_number(result['low'].cast('float'),2).alias('low'),
                                format_number(result['volume'].cast('int'),2).alias('volume')).show()

+-------+--------+--------+--------+--------+-------------+
summary| Open| close| High| low| volume|
+-------+--------+--------+--------+--------+-------------+
 count|1,258.00|1,258.00|1,258.00|1,258.00| 1,258.00|
 mean| 72.36| 72.39| 72.84| 71.92| 8,222,093.00|
 stddev| 6.77| 6.76| 6.77| 6.74| 4,519,780.00|
 min| 56.39| 56.42| 57.06| 56.30| 2,094,900.00|
 max| 90.80| 90.47| 90.97| 89.25|80,898,100.00|
+-------+--------+--------+--------+--------+-------------+

In [ ]:
# Create a new Dataframe with a column called HV Ratio that is ratio of the High price vs volume of stock traded for the day
df2 = df1.withColumn("HV Ratio", df1['high']/df1['Volume']) # here dividing the column with other column
df2.select("HV Ratio").show()

+--------------------+
 HV Ratio|
+--------------------+
4.819714653321546E-6|
6.290848613094555E-6|
4.669412994783916E-6|
7.367338463826307E-6|
8.915604778943901E-6|
8.644477436914568E-6|
9.351828421515645E-6|
 8.29141562102703E-6|
7.712212102001476E-6|
7.071764823529412E-6|
1.015495466386981E-5|
6.576354146362592...|
 5.90145296180676E-6|
8.547679455011844E-6|
8.420709512685392E-6|
1.041448341728929...|
8.316075414862431E-6|
9.721183814992126E-6|
8.029436027707578E-6|
6.307432259386365E-6|
+--------------------+
only showing top 20 rows

In [ ]:
# what day had the peak high in price
df1.orderBy(df1['High'].desc()).head(1)[0][0]

Out[21]: '2015-01-13'

In [ ]:
# what is the max and min of volume column
from pyspark.sql.functions import min, max
df1.select(max('volume'),min('volume')).show()

+-----------+-----------+
max(volume)|min(volume)|
+-----------+-----------+
 80898100| 2094900|
+-----------+-----------+

In [ ]:
# how many days was the close lower than 60 $
# df1.filter('Close < 60').count()

# if wanted to see the result in the form of the dataframe
from pyspark.sql.functions import count
result = df1.filter(df1['close'] < 60)
result.select(count('Close')).show()

+------------+
count(Close)|
+------------+
 81|
+------------+

In [ ]:
# what % of time was the high > 80$ i.e (Number of days high > 80)/(Total no of days)
# df1.count() - Total no of days - o/p - 1258
(df1.filter(df1['high'] > 80).count() / df1.count())*100

Out[28]: 9.141494435612083

In [ ]:
# what is the Pearson coorealtion between high and volume
from pyspark.sql.functions import corr
df1.select(corr('High','Volume')).show()

+-------------------+
 corr(High, Volume)|
+-------------------+
-0.3384326061737161|
+-------------------+

In [ ]:
# what is the max high per year
from pyspark.sql.functions import year
yeardf = df1.withColumn("Year",year(df1['Date'])) # so here we are calling year function and passing the date column in it and not the date string
max_df = yeardf.groupBy('year').max()
max_df.select('year','max(high)').show()

+----+---------+
year|max(high)|
+----+---------+
2015|90.970001|
2013|81.370003|
2014|88.089996|
2012|77.599998|
2016|75.190002|
+----+---------+

In [ ]:
# Across all year, what is the average close price for each month
from pyspark.sql.functions import month
month_df = df1.withColumn("month",month(df1['Date']))
month_avg = month_df.select('Month','close').groupBy('Month').mean()
month_avg.select('month','avg(close)').orderBy('Month').show()

+-----+-----------------+
month| avg(close)|
+-----+-----------------+
 1|71.44801958415842|
 2| 71.306804443299|
 3|71.77794377570092|
 4|72.97361900952382|
 5|72.30971688679247|
 6| 72.4953774245283|
 7|74.43971943925233|
 8|73.02981855454546|
 9|72.18411785294116|
 10|71.57854545454543|
 11| 72.1110893069307|
 12|72.84792478301885|
+-----+-----------------+

In [ ]:
## SECTION 10 - ONLY CONISIST OF THEORY
## SECTION 11 - MACHINE LEARNING WITH PYSPARK (MLib)

# Linear regression 
spark = SparkSession.builder.appName("lrex").getOrCreate()
from pyspark.ml.regression import LinearRegression

In [ ]:
training = spark.read.format('libsvm').load('dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/sample_linear_regression_data.txt')  

In [ ]:
training.show()
# label column must have some sort of numerical label - Either the numerical regression value or numerical matching the classification grouping.
# feature has the vector, a vector of all the features that brelong to that row

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [ ]:
# create the instance of the model - lr is the instance of linear regression
# LinearRegression takes 3 parameters, first one is the featuresCol which tell the LinearRegression model what is the feature column called and by deafult is called as 'features'('features' in above table), another is labelCol which is called 'label' by default and third is predictionCol which is called 'prediction' by default. All this default value change according to the above column name and datasets used.  
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [ ]:
# Next is to fit or train the model 
lrmodel = lr.fit(training)

In [ ]:
lrmodel.coefficients # o/p = DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956]) - Here the coefficients (eg : 0.0073) denotes features of the model
lrmodel.intercept #o/p = 0.14228558260358093

Out[47]: 0.14228558260358093

In [ ]:
training_summary = lrmodel.summary
training_summary.r2 # It tells how much the variance has model 
training_summary.rootMeanSquaredError

Out[49]: 10.16309157133015

In [ ]:
# On above we have directly trained on the entire dataset, but now we will divide it into train and test dataset
all_data = spark.read.format('libsvm').load('dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/sample_linear_regression_data.txt')  

In [ ]:
# now we would split the data into train and test with the help random split with 70:30 - train : test
split_object = all_data.randomSplit([0.7,0.3])
split_object # It is the list of 2 dataframes where first dataframe has 70% of data of total data and 2nd dataframe has 30% of total data

# Can be written as - 
train_data, test_data = all_data.randomSplit([0.7,0.3])
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 342|
 mean|-0.0955999158266894|
 stddev| 10.616361595268224|
 min|-28.571478869743427|
 max| 27.78383192005107|
+-------+-------------------+

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 159|
 mean| 1.015072145486905|
 stddev| 9.63325598917348|
 min|-28.046018037776633|
 max| 27.111027963108548|
+-------+-------------------+

In [ ]:
correct_model = lr.fit(train_data) # training on the train_data
test_result = correct_model.evaluate(test_data)

In [ ]:
# residuals means what did the model expected vs what the label we knew for the test data. So we are comparing our predictions to the label already present in the test data
test_result.residuals.show()
test_result.rootMeanSquaredError

+-------------------+
 residuals|
+-------------------+
-26.630926720514275|
-24.473846684378046|
 -21.11532036058513|
-16.066438004096486|
-16.935902699773255|
-17.934880982208544|
-14.753087362263374|
-15.563508433153578|
 -13.95271776994644|
-14.379114306368393|
-17.098338285983004|
-10.647436357965802|
 -9.363707039808954|
 -9.39062678446345|
-10.928169402136906|
-13.314539715197357|
-11.942779118777802|
-10.415704630121537|
 -6.83219249951915|
 -9.880075318292917|
+-------------------+
only showing top 20 rows

Out[60]: 9.896254296204289

In [ ]:
unlabeled_data = test_data.select('features')
unlabeled_data.show() # It will display only the feature col

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [ ]:
# Creating the dataframe called predictions in which we transform on this unlabeled_data in which we get the prediction value for the corrosponding features. 
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
 features| prediction|
+--------------------+--------------------+
(10,[0,1,2,3,4,5,...| -1.4150913172623572|
(10,[0,1,2,3,4,5,...| -2.262360498223678|
(10,[0,1,2,3,4,5,...| 2.2693978876865466|
(10,[0,1,2,3,4,5,...| -1.4277623527868555|
(10,[0,1,2,3,4,5,...| 0.9843901339786834|
(10,[0,1,2,3,4,5,...| 2.600113502286202|
(10,[0,1,2,3,4,5,...| -0.5578932271529157|
(10,[0,1,2,3,4,5,...| 0.7413555234023884|
(10,[0,1,2,3,4,5,...|-0.02341316120626233|
(10,[0,1,2,3,4,5,...| 1.2257787000028622|
(10,[0,1,2,3,4,5,...| 4.176115182612582|
(10,[0,1,2,3,4,5,...| -1.4829168543221274|
(10,[0,1,2,3,4,5,...| -2.730644238726304|
(10,[0,1,2,3,4,5,...| -2.4667235809659758|
(10,[0,1,2,3,4,5,...| -0.6876058628787206|
(10,[0,1,2,3,4,5,...| 2.7144093732883245|
(10,[0,1,2,3,4,5,...| 1.4536219954049028|
(10,[0,1,2,3,4,5,...| 0.12704737773282862|
(10,[0,1,2,3,4,5,...| -3.4012470874340037|
(10,[0,1,2,3,4,5,...|-0.01208060953330...|
+--------------------+--------------------+
only showing top 20 rows

In [ ]:
## SECTION 11 - LINEAR REGRESSION WITH ECOMMERCE_CUSTOMER EXAMPLE 

spark = SparkSession.builder.appName("lr_example").getOrCreate()
from pyspark.ml.regression import LinearRegression

In [ ]:
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/Ecommerce_Customers.csv", inferSchema = True, header = True)
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [ ]:
# Directly printing out the value
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005

In [ ]:
# Now will set the dataframe for machine learning - Using mllib with python and spark. To import vector and vector assembler
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

# Want to grab the numerical data above as the feature and let say to grab the 'Yearly Amount Spent' i.e '587.9510539684005' as feature to predict. 

In [ ]:
data.columns

Out[69]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [ ]:
# To use the VectorAssembler - inputCols is the list of string of actual col that we want to take in i.e the all of the feature columns
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol = 'features')
output = assembler.transform(data)
output.printSchema() # so here features is being added which contains ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership']
output.select('features').show()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

+--------------------+
 features|
+--------------------+
[34.4972677251122...|
[31.9262720263601...|
[33.0009147556426...|
[34.3055566297555...|
[33.3306725236463...|
[33.8710378793419...|
[32.0215955013870...|
[32.7391429383803...|
[33.9877728956856...|
[31.9365486184489...|
[33.9925727749537...|
[33.8793608248049...|
[29.5324289670579...|
[33.1903340437226...|
[32.3879758531538...|
[30.7377203726281...|
[32.1253868972878...|
[32.3388993230671...|
[32.1878120459321...|
[32.6178560628234...|
+--------------------+
only showing top 20 rows

In [ ]:
output.head(1) # So as we can see that features contain the dense vector with 4 value of 1st row of ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'] --> features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826])

Out[78]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [ ]:
final_data = output.select('features','Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [ ]:
# Now to split it into train and the test data
train_data, test_data = final_data.randomSplit([0.7,0.3])
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 368|
 mean| 498.983626470116|
 stddev| 76.34656005665785|
 min| 266.086340948469|
 max| 744.2218671047146|
+-------+-------------------+

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 132|
 mean| 500.235186274945|
 stddev| 87.36191670168888|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [ ]:
lr = LinearRegression(labelCol='Yearly Amount Spent', featuresCol='features')
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 -4.339119242820118|
 11.150817082759772|
 5.399185164686401|
-3.2674397825660435|
 3.6101328793585026|
 19.559142884390496|
 3.10452972967704|
 -6.366519218335156|
 -3.532650916490013|
-14.002810340724182|
-1.5297301969175123|
 6.773944657805885|
 -6.01065747635829|
 -4.294892044519713|
-10.425749407706007|
-2.5758938112239775|
 8.474915516413546|
 -1.950004242153227|
 -2.065588865313316|
-10.444801107601165|
+-------------------+
only showing top 20 rows

In [ ]:
test_results.rootMeanSquaredError
test_results.r2
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [ ]:
# lets ts it on the unlabeled data
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[30.4925366965402...|
[30.7377203726281...|
[31.0472221394875...|
[31.2681042107507...|
[31.3091926408918...|
[31.3123495994443...|
[31.4459724827577...|
[31.4474464941278...|
[31.5171218025062...|
[31.5741380228732...|
[31.5761319713222...|
[31.6548096756927...|
[31.7242025238451...|
[31.7656188210424...|
[31.8093003166791...|
[31.8124825597242...|
[31.8209982016720...|
[31.8530748017465...|
[31.8627411090001...|
[31.8648325480987...|
+--------------------+
only showing top 20 rows

In [ ]:
# Creating the dataframe called predictions in which we transform on this unlabeled_data in which we get the prediction value for the corrosponding features. 
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[30.4925366965402...|286.81036496273464|
[30.7377203726281...| 450.6299251134701|
[31.0472221394875...| 387.098214024335|
[31.2681042107507...|426.73797295638997|
[31.3091926408918...| 429.1105849605751|
[31.3123495994443...| 444.0322751435501|
[31.4459724827577...|481.77243520545153|
[31.4474464941278...| 424.9692613135592|
[31.5171218025062...| 279.4510715668757|
[31.5741380228732...| 558.412082501311|
[31.5761319713222...| 542.7563141862458|
[31.6548096756927...|468.48947906974263|
[31.7242025238451...| 509.3985447643188|
[31.7656188210424...|500.84897368012685|
[31.8093003166791...| 547.1976487705472|
[31.8124825597242...| 395.3862387950212|
[31.8209982016720...| 416.2003654967998|
[31.8530748017465...| 461.2351277045052|
[31.8627411090001...| 558.36373003936|
[31.8648325480987...|450.33608158441484|
+--------------------+------------------+
only showing top 20 rows

In [ ]:
# SECTION 11 - LINEAR REGRESSION CONSULTING PROJECT SOLUTIONS

spark = SparkSession.builder.appName("cruise").getOrCreate()
from pyspark.ml.regression import LinearRegression

In [ ]:
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/cruise_ship_info.csv", inferSchema = True, header = True)
df1.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [ ]:
# As Cruise_line is in the form of the string, we need to convert it into the integer so that our algorithm can understand. Now to find how many cruise line we have in general, we use groupBy method
df1.groupBy('Cruise_line').count().show()

# As there are many Cruise_line so will use string indexer to convert this Cruise_line into number group 
from pyspark.ml.feature import StringIndexer

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [ ]:
# string Indexer is similar to vetor indexer object 
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_cat')
indexed = indexer.fit(df1).transform(df1)
indexed.head(3) # This will create the cruise_cat (an int) for corrosponding 'Cruise_line' and each 'Cruise_line' will have unque Cruise_cat

Out[102]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_cat=1.0)]

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

indexed.columns

Out[104]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat']

In [ ]:
# inputCols is ['Age', 'Tonnage','passengers','length','cabins','passenger_density','Cruise_cat'] and outputCol is 'features' because we want to predict for 'crew'. So when outputed the 'features' will consist of ['Age', 'Tonnage','passengers','length','cabins','passenger_density','Cruise_cat']. 
assembler = VectorAssembler(inputCols=['Age', 'Tonnage','passengers','length','cabins','passenger_density','Cruise_cat'], outputCol='features')
output = assembler.transform(indexed)
output.select('features','crew').show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [ ]:
final_data = output.select('features','crew')
train_data, test_data = final_data.randomSplit([0.7,0.3])
train_data.describe().show()
test_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 109|
 mean|7.888623853211021|
 stddev|3.493084296649572|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 49|
 mean|7.584081632653063|
 stddev|3.553670589666964|
 min| 0.6|
 max| 13.6|
+-------+-----------------+

In [ ]:
from pyspark.ml.regression import LinearRegression
ship_lr = LinearRegression(labelCol='crew') 
train_ship_model = ship_lr.fit(train_data)
ship_result = train_ship_model.evaluate(test_data)

In [ ]:
ship_result.rootMeanSquaredError
ship_result.r2
ship_result.meanAbsoluteError
ship_result.meanSquaredError

Out[118]: 0.45689950086791287

In [ ]:
# To find the coorealation between 2 columns
from pyspark.sql.functions import corr
df1.select(corr('crew','passengers')).show() # So here they are highly coorealated. 

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [ ]:
# SECTION 12 - LOGISTIC REGRESSION - CLASSIFICATION
spark = SparkSession.builder.appName("mylogreg").getOrCreate()
from pyspark.ml.classification import LogisticRegression

In [ ]:
my_data = spark.read.format('libsvm').load('dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/sample_libsvm_data.txt') 
my_data.show() # Binary Classification

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
 0.0|(692,[129,130,131...|
 1.0|(692,[158,159,160...|
 1.0|(692,[99,100,101,...|
 0.0|(692,[154,155,156...|
 0.0|(692,[127,128,129...|
 1.0|(692,[154,155,156...|
 0.0|(692,[153,154,155...|
 0.0|(692,[151,152,153...|
 1.0|(692,[129,130,131...|
 0.0|(692,[154,155,156...|
 1.0|(692,[150,151,152...|
 0.0|(692,[124,125,126...|
 0.0|(692,[152,153,154...|
 1.0|(692,[97,98,99,12...|
 1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows

In [ ]:
# Create the logistic regression model - my_log_reg_model is the instance logistic regression
my_log_reg_model = LogisticRegression() # it is kept as default

fitted_logreg = my_log_reg_model.fit(my_data)
log_summary = fitted_logreg.summary
log_summary.predictions.printSchema()

root
-- label: double (nullable = true)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [ ]:
# here we need to compare that our label match our actual prediction
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[127,128,129...|[19.8534775948018...|[0.99999999761359...| 0.0|
 1.0|(692,[158,159,160...|[-20.377398194812...|[1.41321555124658...| 1.0|
 1.0|(692,[124,125,126...|[-27.401459284607...|[1.25804865162664...| 1.0|
 1.0|(692,[152,153,154...|[-18.862741612462...|[6.42710509302613...| 1.0|
 1.0|(692,[151,152,153...|[-20.483011832789...|[1.27157209228550...| 1.0|
 0.0|(692,[129,130,131...|[19.8506078990887...|[0.99999999760673...| 0.0|
 1.0|(692,[158,159,160...|[-20.337256674744...|[1.47109814708735...| 1.0|
 1.0|(692,[99,100,101,...|[-19.595579753185...|[3.08850168174720...| 1.0|
 0.0|(692,[154,155,156...|[19.2708803215798...|[0.99999999572670...| 0.0|
 0.0|(692,[127,128,129...|[23.6202328360621...|[0.99999999994480...| 0.0|
 1.0|(692,[154,155,156...|[-24.385235147436...|[2.56818872834210...| 1.0|
 0.0|(692,[153,154,155...|[26.3082522491186...|[0.99999999999624...| 0.0|
 0.0|(692,[151,152,153...|[25.8329060319863...|[0.99999999999396...| 0.0|
 1.0|(692,[129,130,131...|[-19.794609138975...|[2.53110684557833...| 1.0|
 0.0|(692,[154,155,156...|[21.0260440948316...|[0.99999999926123...| 0.0|
 1.0|(692,[150,151,152...|[-22.764979942712...|[1.29806018811937...| 1.0|
 0.0|(692,[124,125,126...|[21.5049307194189...|[0.99999999954235...| 0.0|
 0.0|(692,[152,153,154...|[31.9927184226788...|[0.99999999999998...| 0.0|
 1.0|(692,[97,98,99,12...|[-20.521067180106...|[1.22409115654132...| 1.0|
 1.0|(692,[124,125,126...|[-22.245377742546...|[2.18250475445942...| 1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [ ]:
# Passing the evaluate dataframe into evaluator itself i.e to convert the my_data to train and test dataset
lr_train, lr_test = my_data.randomSplit([0.7,0.3])
final_model = LogisticRegression()
fit_final = final_model.fit(lr_train)
predictions_and_labels = fit_final.evaluate(lr_test)

In [ ]:
predictions_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[100,101,102...|[10.4895027746182...|[0.99997217374619...| 0.0|
 0.0|(692,[124,125,126...|[31.8183108232038...|[0.99999999999998...| 0.0|
 0.0|(692,[126,127,128...|[21.8472524371126...|[0.99999999967501...| 0.0|
 0.0|(692,[126,127,128...|[30.5074852148289...|[0.99999999999994...| 0.0|
 0.0|(692,[127,128,129...|[23.9440991840164...|[0.99999999996007...| 0.0|
 0.0|(692,[129,130,131...|[17.8542013258696...|[0.99999998237947...| 0.0|
 0.0|(692,[150,151,152...|[18.1586717468921...|[0.99999998700461...| 0.0|
 0.0|(692,[152,153,154...|[13.9362992505552...|[0.99999911377958...| 0.0|
 0.0|(692,[152,153,154...|[13.9250012113341...|[0.99999910371026...| 0.0|
 0.0|(692,[153,154,155...|[-24.488362918272...|[2.31653632014159...| 1.0|
 0.0|(692,[153,154,155...|[12.0899556766022...|[0.99999438439535...| 0.0|
 0.0|(692,[154,155,156...|[-26.272195067098...|[3.89162204483620...| 1.0|
 1.0|(692,[97,98,99,12...|[-21.278972598194...|[5.73666672183644...| 1.0|
 1.0|(692,[124,125,126...|[-28.997570805025...|[2.54985221833547...| 1.0|
 1.0|(692,[124,125,126...|[-23.123716546968...|[9.06770750032896...| 1.0|
 1.0|(692,[126,127,128...|[-32.481298769871...|[7.82620427105394...| 1.0|
 1.0|(692,[128,129,130...|[-26.129265242606...|[4.48956526136531...| 1.0|
 1.0|(692,[128,129,130...|[-23.492281615255...|[6.27237084025502...| 1.0|
 1.0|(692,[129,130,131...|[-31.454751312751...|[2.18461582472803...| 1.0|
 1.0|(692,[129,130,131...|[-20.400515303195...|[1.38092081288932...| 1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [ ]:
# Now for multiclass classification evaluator and binaryclass classification evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
my_eval = BinaryClassificationEvaluator() # will keep as default as label and raw prediction (This is mentioned in Documentation and video - from 10:40)

In [ ]:
# To evaluate on the prediction dataframe 
my_final_roc = my_eval.evaluate(predictions_and_labels.predictions)
my_final_roc # it means that area under cuver is 0.8928571428571429

Out[137]: 0.8928571428571429

In [ ]:
## SECTION 12 - LOGISTIC REGRESSION ON TITANIC DATASET 

df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/titanic.csv",inferSchema = True, header = True)
df1.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [ ]:
df1.columns
my_cols = df1.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [ ]:
my_final_data = my_cols.na.drop()

In [ ]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

# So first we are going to create the stringIndexer and then we are going to one hot encode that string, this means tht stringIndexer allows to convert every string to number. A example of this is gender indexer i.e male and female
gender_Indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex') # stringIndexer would assign a number to every category of that column

# Now we will one hot encode them, this means the number is converted into the form of array of 0s and 1s in this case of actual category was. Eg : A B C, so for A - Array is [1, 0, 0], for B - Array is [0, 1, 0]
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [ ]:
# One thing good for StringIndexer and OneHotEncoder is that its not necessary to know how many categories before hand for particular column
# Same as above for embarked column
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkedIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkedIndex',outputCol='EmbarkedVec')

In [ ]:
# Now to assemble all this using VectorAssembler
assembler = VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch','Fare','EmbarkedVec'],outputCol='features')

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Now to set the pipeline - Pipeline does is it set the stages for the different steps. If there is complex machine learning task then to set stages. 
from pyspark.ml import Pipeline

In [ ]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

# Now we will set the pipeline by taking stage as the parameter where stage is the list - ['gender_Indexer','gender_encoder', 'embark_indexer','embark_encoder', 'assembler','log_reg_titanic']
pipeline = Pipeline(stages=[gender_Indexer,gender_encoder ,embark_indexer,embark_encoder ,assembler,log_reg_titanic]) # In the last passing the model - log_reg_titanic

In [ ]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3]) # train_data and test_data are 2 dataframes 
fit_model = pipeline.fit(train_data)

In [ ]:
results = fit_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived') # why 'prediction' is because when we transoform the test data, it will automatically call the predicted column the 'prediction' in lower case

results.select('Survived','prediction').show() # For some it will show opposite of 'survived' and 'predicted', but for some it will match

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [ ]:
# So binary classification is used to check for Area under Curve
AUC = my_eval.evaluate(results)
AUC

Out[173]: 0.7901785714285715

In [ ]:
## SECTION 12 - LOGISTIC REGRESSION CONSULTING PROJECT 

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("mylogregconsult").getOrCreate()

In [ ]:
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/customer_churn.csv", inferSchema = True, header = True)

In [ ]:
df1.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-28 04:07:38|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+

In [ ]:
df1.columns
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols = ['Age','Total_Purchase','Account_Manager','Years','Num_Sites'], outputCol = 'features')
output = assembler.transform(df1)

final_data = output.select('features','churn')

In [ ]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

from pyspark.ml.classification import LogisticRegression
lr_churn = LogisticRegression(labelCol='churn') # lr_churn is the instance of the LogisticRegression

In [ ]:
fitted_churn_model = lr_churn.fit(train_churn)

In [ ]:
training_sum = fitted_churn_model.summary
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
summary| churn| prediction|
+-------+-------------------+-------------------+
 count| 644| 644|
 mean|0.17701863354037267|0.13198757763975155|
 stddev| 0.3819811574265452|0.33874036149365794|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [ ]:
# We can use BinaryClassificationEvaluator to evaluate our model. We can set the required column names in rawPredictionCol and labelCol Param and the metric in metricName Param.
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred_and_labels = fitted_churn_model.evaluate(test_churn)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[22.0,11254.38,1....| 0|[4.41887332445740...|[0.98809562293643...| 0.0|
[27.0,8628.8,1.0,...| 0|[5.34755765846954...|[0.99526278623027...| 0.0|
[28.0,11204.23,0....| 0|[1.43245979654457...|[0.80728429125176...| 0.0|
[29.0,12711.15,0....| 0|[5.25994092235063...|[0.99483124319891...| 0.0|
[29.0,13240.01,1....| 0|[6.82578827514549...|[0.99891575686463...| 0.0|
[30.0,8403.78,1.0...| 0|[5.89658721258902...|[0.99725872531005...| 0.0|
[30.0,10183.98,1....| 0|[2.70553272766808...|[0.93735232922516...| 0.0|
[30.0,10960.52,1....| 0|[2.20899199774677...|[0.90105409408611...| 0.0|
[31.0,8688.21,0.0...| 0|[6.53241507263496...|[0.99854662886591...| 0.0|
[31.0,8829.83,1.0...| 0|[4.33546963138388...|[0.98707355862241...| 0.0|
[31.0,9574.89,0.0...| 0|[3.05102144732453...|[0.95482660483267...| 0.0|
[31.0,11743.24,0....| 0|[6.72989878462087...|[0.99880677162552...| 0.0|
[31.0,12264.68,1....| 0|[3.45269235622128...|[0.96931133110195...| 0.0|
[32.0,6367.22,1.0...| 0|[2.77506870011764...|[0.94131362027469...| 0.0|
[32.0,13630.93,0....| 0|[2.05237039081655...|[0.88618691449143...| 0.0|
[33.0,8556.73,0.0...| 0|[3.52423324769135...|[0.97136946939172...| 0.0|
[33.0,10709.39,1....| 0|[6.34469344741537...|[0.99824703729522...| 0.0|
[33.0,12115.91,1....| 0|[2.30855099503339...|[0.90958275698526...| 0.0|
[33.0,12249.96,0....| 0|[5.42493585873842...|[0.99561397325990...| 0.0|
[33.0,13314.19,0....| 0|[2.67448596332742...|[0.93550422615255...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol='churn')
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc # 0.7666666666666666

Out[22]: 0.7666666666666666

In [ ]:
final_lr_model = lr_churn.fit(final_data)

In [ ]:
# Predict on the data file
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/new_customers.csv", inferSchema = True, header = True)

# To check that everything is in same format
df1.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [ ]:
test_new_customers = assembler.transform(df1)
test_new_customers.printSchema() # so it added the feature column at the end 
final_results = final_lr_model.transform(test_new_customers)
final_results.show()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company| features| rawPrediction| probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|[37.0,9935.53,1.0...|[2.22168705251390...|[0.90218018099700...| 0.0|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595046...|[0.00198380445828...| 1.0|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|[-3.7691621189384...|[0.02255110110417...| 1.0|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|[32.0,6487.5,0.0,...|[-5.0956222016539...|[0.00608622642083...| 1.0|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|[32.0,13147.71,1....|[1.10475867224048...|[0.75115067517455...| 0.0|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|[22.0,8445.26,1.0...|[-1.6896019277075...|[0.15582819767621...| 1.0|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+

In [ ]:
## SECTION 13 - TREE METHODS

spark = SparkSession.builder.appName('mytree').getOrCreate()
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

# If we have regression problem then this can be applied to regresson

In [ ]:
my_data = spark.read.format('libsvm').load('dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/sample_libsvm_data.txt') 
my_data.show() 

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
 0.0|(692,[129,130,131...|
 1.0|(692,[158,159,160...|
 1.0|(692,[99,100,101,...|
 0.0|(692,[154,155,156...|
 0.0|(692,[127,128,129...|
 1.0|(692,[154,155,156...|
 0.0|(692,[153,154,155...|
 0.0|(692,[151,152,153...|
 1.0|(692,[129,130,131...|
 0.0|(692,[154,155,156...|
 1.0|(692,[150,151,152...|
 0.0|(692,[124,125,126...|
 0.0|(692,[152,153,154...|
 1.0|(692,[97,98,99,12...|
 1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows

In [ ]:
train_data, test_data = my_data.randomSplit([0.7,0.3])

# To make the instance of the DecisionTreeClassifier and  
dtc = DecisionTreeClassifier()
rtc = RandomForestClassifier()
gbt = GBTClassifier()

In [ ]:
# training on the train_data
dtc_model = dtc.fit(train_data)
rtc_model = rtc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [ ]:
dtc_preds = dtc_model.transform(test_data) 
rtc_preds = rtc_model.transform(test_data) 
gbt_preds = gbt_model.transform(test_data) 

In [ ]:
# with BinaryClassificationEvaluator i would be only able to do precision recall curve and area under ROC curve, but with MulticlassClassificationEvaluator, we can do stuff of BinaryClassificationEvaluator and also the matric like precession, recall and accuracy 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

# accuracy for decision tree classifier
acc_eval.evaluate(dtc_preds) # 1.0 => 100% (Artificial tree, thats why 100%)

Out[43]: 1.0

In [ ]:
# Now suppose to check the importance of the feature - Higher the number, more is the importance 
rtc_model.featureImportances

Out[44]: SparseVector(692, {207: 0.0029, 208: 0.0016, 236: 0.0009, 239: 0.0031, 260: 0.0222, 261: 0.0103, 272: 0.0291, 298: 0.0026, 319: 0.003, 322: 0.0093, 351: 0.0052, 359: 0.0017, 378: 0.0843, 399: 0.0048, 400: 0.0027, 407: 0.0446, 434: 0.1467, 454: 0.0094, 456: 0.0357, 461: 0.0183, 462: 0.05, 466: 0.011, 467: 0.0278, 489: 0.178, 494: 0.0397, 497: 0.0094, 512: 0.0406, 517: 0.0643, 519: 0.0013, 520: 0.0018, 523: 0.039, 539: 0.039, 540: 0.0017, 548: 0.0101, 549: 0.0021, 551: 0.0363, 553: 0.0014, 572: 0.002, 602: 0.0047, 630: 0.0013})

In [ ]:
## SECTION 13 - DECISION TREE AND RANDOM FOREST ON COLLEGE DATASET 

spark = SparkSession.builder.appName('tree').getOrCreate()
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/College.csv", inferSchema = True, header = True)
data.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [ ]:
# data.head(1)
data.columns

Out[49]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [ ]:
# To format out the data using VectorAssembler
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['Apps','Accept','Enroll','Top10perc','Top25perc','F_Undergrad','P_Undergrad','Outstate','Room_Board','Books','Personal','PhD','Terminal','S_F_Ratio','perc_alumni','Expend','Grad_Rate'],outputCol='features')

output = assembler.transform(data)

In [ ]:
# Now to convert the 'private' datatype (which consist of 'yes' or 'no') to 0s and 1s using stringIndexer
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex') # if wanted to use stringindexer for the single column then it is not necessary to use Pipeline, Directly use stringindexer
output_fixed = indexer.fit(output).transform(output)

In [ ]:
output_fixed.printSchema() # features and Privateindex are added 
final_data = output_fixed.select('features','PrivateIndex')
train_data, test_data = final_data.randomSplit([0.7,0.3])

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- features: vector (nullable = true)
-- PrivateIndex: double (nullable = false)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier
from pyspark.ml import Pipeline

In [ ]:
# Now would be using 3 models - DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')

# rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
# if we change the default number of trees from 20 to 150 then AUC will increase
rfc = RandomForestClassifier(numTrees=150,labelCol='PrivateIndex',featuresCol='features')

gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

In [ ]:
# training - 
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [ ]:
# To check on the test data
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# my_binary_eval is the object which is set to the instance of BinaryClassificationEvaluator. As labelCol is not called as label, but privateIndex and hence written 
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

print('DTC')
print(my_binary_eval.evaluate(dtc_preds)) # So this will give the area under curve 

print('RFC')
print(my_binary_eval.evaluate(rfc_preds)) # So this will give the area under curve - RFC will always perform better than DTC
# initially at 20 trees - 0.9717948717948718, then at 150 trees - 0.9783216783216784

DTC
0.9388111888111889
RFC
0.9783216783216784

In [ ]:
gbt_preds.printSchema()
print('GBT')
print(my_binary_eval.evaluate(gbt_preds))

root
-- features: vector (nullable = true)
-- PrivateIndex: double (nullable = false)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

GBT
0.9547785547785552

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')
rfc_acc = acc_eval.evaluate(rfc_preds)
rfc_acc

Out[78]: 0.9354838709677419

In [ ]:
# SECTION 13 - DECISION TREE AND RANDOM FOREST CONSULTING PROJECT 

# Content and important notes - 
# Our main task was to figure out which preservative chemical (A,B,C,D) was having an effect on the dog food being spoiled.
# Many machine learning models produce some sort of coefficient value for each feature involved, indicating their “importance” or predictive power.
# We mentioned that these tree method classifiers had a .featureImportances attribute available! So we can create a model, fit it on all the data, and then check which feature (preservative) was causing the spoilage! 
# .featureImportances returns: a sparse vector where first number is the total number of features and second parameter is the dictionary where we have index number of feature and the value of the key corrosponds to importance of the feature. Eg: SparseVector(4, {0: 0.0026, 1: 0.0089, 2: 0.9686, 3: 0.0199}) - So here important feature is at index 2 giving importance of 96%
# Corresponding to a features column: we have the row in which features we have imported as the dense vector having the presevatives value and finally we have labelled column 'spoiled' i.e Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [ ]:
spark = SparkSession.builder.appName('tree').getOrCreate()
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/dog_food.csv",inferSchema = True, header = True)
data.printSchema()
data.columns

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)

Out[82]: ['A', 'B', 'C', 'D', 'Spoiled']

In [ ]:
# we are going to perform random forest clasifier model on the data itself and for that feature must be in right format, therefore will use vector assenbler
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol='features')
output = assembler.transform(data)

In [ ]:
# we are going to perform random forest clasifier model
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(labelCol='spoiled',featuresCol='features')
final_data = output.select('features','spoiled')

In [ ]:
# Train the classifier
rfc_model = rfc.fit(final_data)
rfc_model.featureImportances # so from this we can come to know that chemical 'C' at index 2 that is causing the spoilage 

Out[88]: SparseVector(4, {0: 0.0189, 1: 0.0141, 2: 0.9384, 3: 0.0287})

In [ ]:
## SECTION 14 - K MEANS AND CLUSTERING

spark = SparkSession.builder.appName('cluster').getOrCreate()
from pyspark.ml.clustering import KMeans
data = spark.read.format('libsvm').load('dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/sample_kmeans_data.txt')  
data.show() # this example set K = 2

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0| (3,[],[])|
 1.0|(3,[0,1,2],[0.1,0...|
 2.0|(3,[0,1,2],[0.2,0...|
 3.0|(3,[0,1,2],[9.0,9...|
 4.0|(3,[0,1,2],[9.1,9...|
 5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+

In [ ]:
# In unsupervised learning we only need features and not the label column 
final_data = data.select('features')
final_data.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [ ]:
# Now to set the value of K (will follow the K given in the documentation example) and the seed value - It is seed value for random number generator. Seed function is used to save the state of a random function, so that it can generate same random numbers on multiple executions of the code on the same machine. In this case the setSeed is used fro the assignment of centriods
# kmeans = KMeans().setK(2).setSeed(1)
kmeans = KMeans().setK(3).setSeed(1) # Set K = 3

# Now to fit the model 
model = kmeans.fit(final_data)

In [ ]:
# To check for within set sum of square errors (WSSSE). We compute the cost of entire dataset using trainingCost. As K value increases WSSSE decreases 
wssse = model.summary.trainingCost
wssse 
final_data.show() # so we have 3 dimension of features - all 3 items/dimensios in tuple 

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [ ]:
# As we have 3 dimension of features, we can expect the centriods to be in 3D
centers = model.clusterCenters()
centers # As we computed the value of K = 2, this means there are 2 centriods and they are this location (x,y,z) - [array([9.1, 9.1, 9.1]), array([0.1, 0.1, 0.1])]

Out[107]: [array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [ ]:
# Now suppose we want to set the data to which centers it belongs to 
results = model.transform(final_data)
results.show() # So it gives the features along with the prediction

+--------------------+----------+
 features|prediction|
+--------------------+----------+
 (3,[],[])| 1|
(3,[0,1,2],[0.1,0...| 1|
(3,[0,1,2],[0.2,0...| 2|
(3,[0,1,2],[9.0,9...| 0|
(3,[0,1,2],[9.1,9...| 0|
(3,[0,1,2],[9.2,9...| 0|
+--------------------+----------+

In [ ]:
## SECTION 14 - CLUSTERING 

# For certain Machine Learning algorithms, it is a good idea to scale your data. Drops in model performance can occur with highly dimensional data, so we’ll practice scaling features using PySpark! there won’t be any confusion matrix or classification test results as this is the unsupervised learning!
spark = SparkSession.builder.appName('cluster').getOrCreate()
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/seeds_dataset.csv", inferSchema = True, header = True)
data.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [ ]:
data.head(1)# This all are the features of actual seed in kernel 
# As there are 3 different varieties of wheat, we are going to cluster with K = 3 
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
data.columns

Out[114]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [ ]:
assembler = VectorAssembler(inputCols=data.columns, outputCol='features') # rather than passing the columns, will pass the data.columns in the inputCols as we are taking all columns 
final_data = assembler.transform(data)
final_data.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [ ]:
# To scale the data
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features',outputCol='Scaledfeature') # Synatx is similar as VectorAssembler
scaler_model = scaler.fit(final_data) # When we fit the scaler to the final data, it collects the stddev information and then it set to the scaler model and then the scaler model is transformed to final_data when called the transform below
final_data = scaler_model.transform(final_data)
final_data.show()
# Here there is no huge change between features and scaled features, but it is very important for the large datasets 

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+--------------------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove| features| Scaledfeatures| Scaledfeature|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+--------------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|[15.26,14.84,0.87...|[5.24452795332028...|[5.24452795332028...|
14.88| 14.57| 0.8811| 5.553999999999999| 3.333| 1.018| 4.956|[14.88,14.57,0.88...|[5.11393027165175...|[5.11393027165175...|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|[14.29,14.09,0.90...|[4.91116018695588...|[4.91116018695588...|
13.84| 13.94| 0.8955| 5.324|3.3789999999999996| 2.259| 4.805|[13.84,13.94,0.89...|[4.75650503761158...|[4.75650503761158...|
16.14| 14.99| 0.9034|5.6579999999999995| 3.562| 1.355| 5.175|[16.14,14.99,0.90...|[5.54696468981581...|[5.54696468981581...|
14.38| 14.21| 0.8951| 5.386| 3.312| 2.4619999999999997| 4.956|[14.38,14.21,0.89...|[4.94209121682475...|[4.94209121682475...|
14.69| 14.49| 0.8799| 5.563| 3.259| 3.5860000000000003| 5.218999999999999|[14.69,14.49,0.87...|[5.04863143081749...|[5.04863143081749...|
14.11| 14.1| 0.8911| 5.42| 3.302| 2.7| 5.0|[14.11,14.1,0.891...|[4.84929812721816...|[4.84929812721816...|
16.63| 15.46| 0.8747| 6.053| 3.465| 2.04| 5.877000000000001|[16.63,15.46,0.87...|[5.71536696354628...|[5.71536696354628...|
16.44| 15.25| 0.888|5.8839999999999995| 3.505| 1.969|5.5329999999999995|[16.44,15.25,0.88...|[5.65006812271202...|[5.65006812271202...|
15.26| 14.85| 0.8696|5.7139999999999995| 3.242| 4.543| 5.314|[15.26,14.85,0.86...|[5.24452795332028...|[5.24452795332028...|
14.03| 14.16| 0.8796| 5.438| 3.201| 1.7169999999999999| 5.001|[14.03,14.16,0.87...|[4.82180387844584...|[4.82180387844584...|
13.89| 14.02| 0.888| 5.439| 3.199| 3.986| 4.738|[13.89,14.02,0.88...|[4.77368894309428...|[4.77368894309428...|
13.78| 14.06| 0.8759| 5.479| 3.156| 3.136| 4.872|[13.78,14.06,0.87...|[4.73588435103234...|[4.73588435103234...|
13.74| 14.05| 0.8744| 5.482| 3.114| 2.932| 4.825|[13.74,14.05,0.87...|[4.72213722664617...|[4.72213722664617...|
14.59| 14.28| 0.8993| 5.351| 3.333| 4.185| 4.781000000000001|[14.59,14.28,0.89...|[5.01426361985209...|[5.01426361985209...|
13.99| 13.83| 0.9183| 5.119| 3.383| 5.234| 4.781000000000001|[13.99,13.83,0.91...|[4.80805675405968...|[4.80805675405968...|
15.69| 14.75| 0.9058| 5.527| 3.514| 1.599| 5.046|[15.69,14.75,0.90...|[5.39230954047151...|[5.39230954047151...|
 14.7| 14.21| 0.9153| 5.205| 3.466| 1.767| 4.649|[14.7,14.21,0.915...|[5.05206821191403...|[5.05206821191403...|
12.72| 13.57| 0.8686| 5.226| 3.049| 4.102| 4.914|[12.72,13.57,0.86...|[4.37158555479908...|[4.37158555479908...|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [ ]:
kmeans = KMeans(featuresCol='Scaledfeature',k=3)
model = kmeans.fit(final_data) # kmeans object is fitted to final data

In [ ]:
print('WSSSE')
wssse = model.summary.trainingCost
wssse 

centers = model.clusterCenters()
centers

WSSSE
Out[136]: [array([ 4.93382436, 10.94691274, 37.30542404, 12.41332714, 8.60366812,
 1.82917353, 10.40106154]),
 array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446, 9.7892399 ,
 2.41585013, 12.29286107]),
 array([ 4.06660859, 10.14191893, 35.84098009, 11.81592066, 7.52397236,
 3.1823335 , 10.39801233])]

In [ ]:
model.transform(final_data).show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+--------------------+----------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove| features| Scaledfeatures| Scaledfeature|prediction|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+--------------------+----------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|[15.26,14.84,0.87...|[5.24452795332028...|[5.24452795332028...| 0|
14.88| 14.57| 0.8811| 5.553999999999999| 3.333| 1.018| 4.956|[14.88,14.57,0.88...|[5.11393027165175...|[5.11393027165175...| 0|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|[14.29,14.09,0.90...|[4.91116018695588...|[4.91116018695588...| 0|
13.84| 13.94| 0.8955| 5.324|3.3789999999999996| 2.259| 4.805|[13.84,13.94,0.89...|[4.75650503761158...|[4.75650503761158...| 0|
16.14| 14.99| 0.9034|5.6579999999999995| 3.562| 1.355| 5.175|[16.14,14.99,0.90...|[5.54696468981581...|[5.54696468981581...| 0|
14.38| 14.21| 0.8951| 5.386| 3.312| 2.4619999999999997| 4.956|[14.38,14.21,0.89...|[4.94209121682475...|[4.94209121682475...| 0|
14.69| 14.49| 0.8799| 5.563| 3.259| 3.5860000000000003| 5.218999999999999|[14.69,14.49,0.87...|[5.04863143081749...|[5.04863143081749...| 0|
14.11| 14.1| 0.8911| 5.42| 3.302| 2.7| 5.0|[14.11,14.1,0.891...|[4.84929812721816...|[4.84929812721816...| 0|
16.63| 15.46| 0.8747| 6.053| 3.465| 2.04| 5.877000000000001|[16.63,15.46,0.87...|[5.71536696354628...|[5.71536696354628...| 1|
16.44| 15.25| 0.888|5.8839999999999995| 3.505| 1.969|5.5329999999999995|[16.44,15.25,0.88...|[5.65006812271202...|[5.65006812271202...| 0|
15.26| 14.85| 0.8696|5.7139999999999995| 3.242| 4.543| 5.314|[15.26,14.85,0.86...|[5.24452795332028...|[5.24452795332028...| 0|
14.03| 14.16| 0.8796| 5.438| 3.201| 1.7169999999999999| 5.001|[14.03,14.16,0.87...|[4.82180387844584...|[4.82180387844584...| 0|
13.89| 14.02| 0.888| 5.439| 3.199| 3.986| 4.738|[13.89,14.02,0.88...|[4.77368894309428...|[4.77368894309428...| 0|
13.78| 14.06| 0.8759| 5.479| 3.156| 3.136| 4.872|[13.78,14.06,0.87...|[4.73588435103234...|[4.73588435103234...| 0|
13.74| 14.05| 0.8744| 5.482| 3.114| 2.932| 4.825|[13.74,14.05,0.87...|[4.72213722664617...|[4.72213722664617...| 0|
14.59| 14.28| 0.8993| 5.351| 3.333| 4.185| 4.781000000000001|[14.59,14.28,0.89...|[5.01426361985209...|[5.01426361985209...| 0|
13.99| 13.83| 0.9183| 5.119| 3.383| 5.234| 4.781000000000001|[13.99,13.83,0.91...|[4.80805675405968...|[4.80805675405968...| 0|
15.69| 14.75| 0.9058| 5.527| 3.514| 1.599| 5.046|[15.69,14.75,0.90...|[5.39230954047151...|[5.39230954047151...| 0|
 14.7| 14.21| 0.9153| 5.205| 3.466| 1.767| 4.649|[14.7,14.21,0.915...|[5.05206821191403...|[5.05206821191403...| 0|
12.72| 13.57| 0.8686| 5.226| 3.049| 4.102| 4.914|[12.72,13.57,0.86...|[4.37158555479908...|[4.37158555479908...| 2|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [ ]:
## SECTION 14 - KMEANS AND CLUSTERING CONSULTING PROJECT 
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/hack_data.csv", inferSchema = True, header = True)
data.head(1)

Out[140]: [Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
data.columns

Out[142]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [ ]:
# Location column is useless because in the question it is written that it is connected through VPN's 
assembler = VectorAssembler(inputCols=['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used','Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed'], outputCol='features')
final_data = assembler.transform(data) # assembler object is transforming the actual data 
final_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

In [ ]:
# to scale the data
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='Scaledfeature')
scaler_model = scaler.fit(final_data)
cluster_final_data = scaler_model.transform(final_data)

In [ ]:
# As we want to predict 2 hackers or 3 hackers - We will do for both K = 2 and K = 3 
kmeans2 = KMeans(featuresCol='Scaledfeature',k=2)
kmeans3 = KMeans(featuresCol='Scaledfeature',k=3)
model_k2 = kmeans2.fit(cluster_final_data)
model_k3 = kmeans3.fit(cluster_final_data)

In [ ]:
# Now according to key fact, the forensic engineer knows that the hackers trade off attacks, Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks, so we will use this info to find if there were 2 hackers or 3 hackers 
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

# so from the results we can conclude that we do not have equal split when there is K = 3, but with K = 2 there is equal split. therefore there is only 2 Hackers 

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [ ]:
## SECTION 15 - RECOMMENDER SYSTEM
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

# We would be importing ALS (Alternating least sqaure) - For Details, refer Udemy PPT and Google
from pyspark.ml.recommendation import ALS
# For Evaluation we will use Regression Evaluator
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/movielens_ratings.csv", inferSchema = True, header = True)
data.show()
data.describe().show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

+-------+------------------+------------------+------------------+
summary| movieId| rating| userId|
+-------+------------------+------------------+------------------+
 count| 1501| 1501| 1501|
 mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
 stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
 min| 0| 1.0| 0|
 max| 99| 5.0| 29|
+-------+------------------+------------------+------------------+

In [ ]:
# splitting to training and testing dataset
train_data, test_data = data.randomSplit([0.8,0.2])

In [ ]:
# Now to create the Alternating least sqaure model - Would be setting maxIter = 5, regression parameter = 0.01, then we need 3 columns which we alwys need i.e userCol, itemcol, ratingCol
als = ALS(maxIter = 5, regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')
model = als.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
predictions.show() 
# To interpret the table - first row -> User ID 4 watched the movie ID 31 and they gave the rating 1.0, but we predicted according to prevous user rating ,this user would be giving the rating of ~2.5 stars. third row - User ID 12 watched the movie ID 31 and they gave the rating 4.0, but we predicted according to prevous user rating ,this user would be giving the rating of ~(-0.3) stars. 
# As we are taking prediction as the continous value, and hence prediction can be negative value 

+-------+------+------+-----------+
movieId|rating|userId| prediction|
+-------+------+------+-----------+
 31| 1.0| 4| 2.4268272|
 31| 3.0| 8| 2.3796675|
 31| 4.0| 12|-0.03163056|
 31| 1.0| 13| 1.5536816|
 31| 3.0| 14| 2.36169|
 31| 2.0| 25| 0.90101004|
 31| 1.0| 26| 2.764635|
 85| 3.0| 6| 2.5745964|
 85| 1.0| 13| 1.2363474|
 85| 1.0| 15| 0.7540424|
 85| 2.0| 20| 2.5667028|
 65| 2.0| 15| 1.4227293|
 53| 1.0| 6| 2.8187125|
 53| 1.0| 9| 3.8860528|
 53| 3.0| 14| 4.938653|
 53| 2.0| 19| 2.9043036|
 78| 1.0| 4| 1.1129671|
 78| 1.0| 12| 0.77230066|
 78| 1.0| 13| 1.1209471|
 78| 1.0| 17| 0.08372226|
+-------+------+------+-----------+
only showing top 20 rows

In [ ]:
# Now to evaluate. We will create the evaluator object taking metricName as 'root mean sqare error(rmse)' 
evaluator = RegressionEvaluator(labelCol='rating',predictionCol='prediction',metricName='rmse')
root_error = evaluator.evaluate(predictions) # This means how far was the rating from the prediction conidering the rmse
root_error

Out[12]: 1.8215280062972696

In [ ]:
# For the single user 
# test_data.filter(test_data['userId']==11).show()
single_user = test_data.filter(test_data['userId']==11).select(['movieId','userId'])
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 11| 11|
 12| 11|
 19| 11|
 21| 11|
 27| 11|
 30| 11|
 39| 11|
 48| 11|
 81| 11|
+-------+------+

In [ ]:
# Now to predict that it is good to watch that particular movie by user 11
recommendations = model.transform(single_user)
recommendations.orderBy('prediction',ascending=False).show() # It gives the recommendations in descending order of prediction 

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 30| 11| 5.418488|
 19| 11| 2.4480557|
 12| 11| 2.2748916|
 21| 11| 1.8056031|
 48| 11| 1.4893379|
 81| 11| 0.82324404|
 11| 11| 0.5711174|
 39| 11|-0.09058407|
 27| 11| -0.51795|
+-------+------+-----------+

In [ ]:
# SECTION 16 - NLP 
spark = SparkSession.builder.appName('nlp').getOrCreate()

# Tokenization - This means taking out the sentence and breaking it to the indiviuals word 
# Regular expression tokenizer - It is more advanced tokenization based on regular expression. It uses the parameter as the eliminator otr spaces to split the sentences 
from pyspark.ml.feature import Tokenizer,RegexTokenizer
from pyspark.sql.functions import col,udf # col -> column, udf -> user defined functions

In [ ]:
from pyspark.sql.types import IntegerType

# Passing out the list of tuples (sentences) as the input - the )th and 1st are having whitespaces, and 2nd is having ',' for splitting. For the 2nd argument we would be passing the list of the column names
sen_df = spark.createDataFrame([(0,'Hi I heard about spark'),(1,'I wish java could use case classes'),(2,'Logistic,regression,models,are,neat')],['id','sentence'])
sen_df.show()

+---+--------------------+
 id| sentence|
+---+--------------------+
 0|Hi I heard about ...|
 1|I wish java could...|
 2|Logistic,regressi...|
+---+--------------------+

In [ ]:
# will create the instance of the tokenizer class
tokenizer = Tokenizer(inputCol='sentence',outputCol='word')

regrex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='word')
# We can even pass the 3rd parameter i.e the pattern (Eg: ',') for splitting of the sentence 
regrex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='word',pattern='\\W')

In [ ]:
# Now will define the user defined function (udf calls) - 2 arguments -> lambda function return the len of words (count of words) and second expression is to define type - IntegerType
# About lambda function -  An anonymous function means that a function is without a name. As we already know that the def keyword is used to define a normal function in Python. Similarly, the lambda keyword is used to define an anonymous function in Python. 'Syntax: lambda arguments: expression'. This function can have any number of arguments but only one expression, which is evaluated and returned. Lambda definition does not include a “return” statement, it always contains an expression that is returned. So here in this case lambda function is going to take in the words and return the len of the words 
count_tokens = udf(lambda words : len(words),IntegerType())

In [ ]:
tokenized = tokenizer.transform(sen_df)
tokenized.show() # word column is the list of the tokens 
tokenized.withColumn('token',count_tokens(col('word'))).show()
# this shows that 2nd list is not divided as there were ',' and not space in between and hence entire sentence is considered as the single word 

+---+--------------------+--------------------+
 id| sentence| word|
+---+--------------------+--------------------+
 0|Hi I heard about ...|[hi, i, heard, ab...|
 1|I wish java could...|[i, wish, java, c...|
 2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+

+---+--------------------+--------------------+-----+
 id| sentence| word|token|
+---+--------------------+--------------------+-----+
 0|Hi I heard about ...|[hi, i, heard, ab...| 5|
 1|I wish java could...|[i, wish, java, c...| 7|
 2|Logistic,regressi...|[logistic,regress...| 1|
+---+--------------------+--------------------+-----+

In [ ]:
# So now will use the regres tokenizer - With this the token count is correct for the 2nd sentence 
rg_tokenized = regrex_tokenizer.transform(sen_df)
rg_tokenized.withColumn('token',count_tokens(col('word'))).show() 

+---+--------------------+--------------------+-----+
 id| sentence| word|token|
+---+--------------------+--------------------+-----+
 0|Hi I heard about ...|[hi, i, heard, ab...| 5|
 1|I wish java could...|[i, wish, java, c...| 7|
 2|Logistic,regressi...|[logistic, regres...| 5|
+---+--------------------+--------------------+-----+

In [ ]:
# stopwordremover - This are the word which appear frequently in the sentences and does not carry out that much meaning
from pyspark.ml.feature import StopWordsRemover
sentenceDataFrame = spark.createDataFrame([(0,['I','saw','the','green','horse']),(1,['Marry','had','a','little','lamp'])],['id','tokens'])
sentenceDataFrame.show()

+---+--------------------+
 id| tokens|
+---+--------------------+
 0|[I, saw, the, gre...|
 1|[Marry, had, a, l...|
+---+--------------------+

In [ ]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')
remover.transform(sentenceDataFrame).show() # The words like I,the,had,a are omitted. 

# LEFT - WITH N-GRAMS

+---+--------------------+--------------------+
 id| tokens| filtered|
+---+--------------------+--------------------+
 0|[I, saw, the, gre...| [saw, green, horse]|
 1|[Marry, had, a, l...|[Marry, little, l...|
+---+--------------------+--------------------+

In [ ]:
# Term frequency - Inverse document frequency (TF-IDF) -> improve on Bag of Words by adjusting word counts based on their frequency in corpus (the group of all the documents)
# Term Frequency - Importance of the term within that document -> TF(x,y) = Number of occurrences of term x in document y
# Inverse Document Frequency - Importance of the term in the corpus -> IDF(t) = log(N/dfx) where N = total number of documents and dfx = number of documents with the term
# HashingTF is imported for term frequency, IDF for Inverse Document Frequency
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
sen_df = spark.createDataFrame([(0.0,'Hi I heard about spark'),(0.0,'I wish java could use case classes'),(1.0,'Logistic regression models are neat')],['label','sentence'])
sen_df.show()

+-----+--------------------+
label| sentence|
+-----+--------------------+
 0.0|Hi I heard about ...|
 0.0|I wish java could...|
 1.0|Logistic regressi...|
+-----+--------------------+

In [ ]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
words_data = tokenizer.transform(sen_df)
words_data.show()

+-----+--------------------+--------------------+
label| sentence| words|
+-----+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|
 0.0|I wish java could...|[i, wish, java, c...|
 1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+

In [ ]:
# Now to deal with term frequency 
hashing_tf = HashingTF(inputCol='words',outputCol='rawfeatures')
featurized_data = hashing_tf.transform(words_data)

In [ ]:
# Now to apply IDF 
idf = IDF(inputCol='rawfeatures',outputCol='features')
idf_model = idf.fit(featurized_data)

In [ ]:
featurized_data.show()
rescaled_data = idf_model.transform(featurized_data)
rescaled_data.select('label','features').show(truncate=False)

+-----+--------------------+--------------------+--------------------+
label| sentence| words| rawfeatures|
+-----+--------------------+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[18700,19...|
 0.0|I wish java could...|[i, wish, java, c...|(262144,[19036,20...|
 1.0|Logistic regressi...|[logistic, regres...|(262144,[46243,58...|
+-----+--------------------+--------------------+--------------------+

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
label|features |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
0.0 |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453]) |
0.0 |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
1.0 |(262144,[46243,58267,91006,160975,190884],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453]) |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [ ]:
# Now coming to count Vectorizer and Count Vectorizer model - It helps in converting the collections of text document into vector of word counts
from pyspark.ml.feature import CountVectorizer
df = spark.createDataFrame([(0,'a b c'.split(" ")),(1,'a b b c a'.split(" "))],['id','words'])
df.show()

+---+---------------+
 id| words|
+---+---------------+
 0| [a, b, c]|
 1|[a, b, b, c, a]|
+---+---------------+

In [ ]:
# For CountVectorizer we use the inputCol, outputCol, vocabSize = 3 (beacuse as shown in the input we have 3 unique vocab size) and minDF is the optional parameter and what it does is it effects the fitting process by specifying the minimum number of documents a term must appear in to be included in the voacb, so suppose if the term/word is appearing in only one document and minDF=2 then it will not appear in final Vocabulary. 
cv = CountVectorizer(inputCol='words',outputCol='features',vocabSize=3, minDF=2.0)
model = cv.fit(df)

In [ ]:
result = model.transform(df)
result.show(truncate=False) 
# It shows that in id=1 as a is appearing 2 times, hence in count in features it is written 2.0, same for b and c. This is the 'bag of words' method 

+---+---------------+-------------------------+
id |words |features |
+---+---------------+-------------------------+
0 |[a, b, c] |(3,[0,1,2],[1.0,1.0,1.0])|
1 |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

In [ ]:
## SECTION 16 - SPAM DECTECTION FILTER USING PYTHON AND SPARK

# spam detection filter using python and spark - NLP
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/nisargmehta2000@gmail.com/SMSSpamCollection", inferSchema = True, sep='\t') # as it is giving out the wrong header therefore will remove header = True
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

In [ ]:
# Firstly we had created the user defined function for calculating the count of words, but we can directly use the 'length' functions
from pyspark.sql.functions import length
data = data.withColumn('length',length(data['text']))
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [ ]:
# To differnciate the ham and spam mails with the help of the length, so from below with the help of avg b/w the length of ham and spam there is the difference between ham and spam messages
data.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [ ]:
# this is form of NLP tools
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

tokenizer = Tokenizer(inputCol='text', outputCol='token_test') # Step1 - Converting the sentences into words
stop_remove = StopWordsRemover(inputCol='token_test', outputCol='stop_token') #step2 - Removing the unwanted words (repeative words)
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec') #step3 - It helps in converting the collections of text document into vector of word counts
idf = IDF(inputCol='c_vec', outputCol='tf_idf') # step4 - apply the tf_idf
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label') #step5 - To convert the ham and spam as a string to index (int)

In [ ]:
# tf_idf is what we got as the output from 4th step from above and we are combining with length col that we prev calculated - finally we are outputting as the feature column 
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [ ]:
# Biuld the model - Classification model - Going to use Navie based which is the classic model to use NLP
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes() # Creating nb object 

In [ ]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[tokenizer,stop_remove,count_vec,idf,ham_spam_to_numeric,clean_up])
cleaner = data_prep_pipe.fit(data)
clean_data = cleaner.transform(data)

In [ ]:
clean_data = clean_data.select('label','features')
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [ ]:
# Now to train and test the data
train_data, test_data = clean_data.randomSplit([0.7,0.3])
spam_detector = nb.fit(train_data)

In [ ]:
test_results = spam_detector.transform(test_data)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,3,9,1...|[-573.08835749228...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-873.95742812119...|[1.0,2.9649021371...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-1171.6426529057...|[1.0,5.1497547762...| 0.0|
 0.0|(13424,[0,1,11,32...|[-868.36748032783...|[1.0,3.6385648521...| 0.0|
 0.0|(13424,[0,1,14,31...|[-216.92864328604...|[1.0,8.7138065557...| 0.0|
 0.0|(13424,[0,1,14,78...|[-691.08157937243...|[1.0,2.3399270281...| 0.0|
 0.0|(13424,[0,1,20,27...|[-968.05512069082...|[1.0,2.7771567289...| 0.0|
 0.0|(13424,[0,1,72,10...|[-715.65093388575...|[0.99999999999998...| 0.0|
 0.0|(13424,[0,1,874,1...|[-97.824792892547...|[0.99999997080170...| 0.0|
 0.0|(13424,[0,1,3657,...|[-144.11325601150...|[0.00328495529375...| 1.0|
 0.0|(13424,[0,2,3,8,2...|[-1585.9602051331...|[1.0,4.3897940519...| 0.0|
 0.0|(13424,[0,2,4,7,2...|[-516.52414549753...|[1.0,1.2355959478...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[-1420.2329807604...|[1.0,1.5426409398...| 0.0|
 0.0|(13424,[0,2,4,11,...|[-1126.2024653362...|[1.0,4.9461966808...| 0.0|
 0.0|(13424,[0,2,4,40,...|[-1579.8039277163...|[0.99999196359776...| 0.0|
 0.0|(13424,[0,2,4,128...|[-640.36896628797...|[1.0,5.2622246686...| 0.0|
 0.0|(13424,[0,2,7,31,...|[-670.36172967697...|[1.0,2.5589078734...| 0.0|
 0.0|(13424,[0,2,7,43,...|[-594.33680467846...|[1.0,3.2284157456...| 0.0|
 0.0|(13424,[0,2,7,60,...|[-1334.7193752061...|[1.0,1.7425092899...| 0.0|
 0.0|(13424,[0,2,7,114...|[-456.30081350342...|[1.0,1.3513991904...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [ ]:
# So as to compare the label with the prediction and for this will use the multiclass classification evaluation technique
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
acc

Out[92]: 0.9176206379796041

In [ ]:
print('COMPLETED THE COUSE OF PYTHON AND SPARK FOR BIG-DATA WITH PYSPARK ON 19th FEBRUARY 2021')

COMPLETED THE COUSE OF PYTHON AND SPARK FOR BIG-DATA WITH PYSPARK ON 19th FEBRUARY 2021